<a href="https://colab.research.google.com/github/YTCX-hub/CSU-Thesis-LaTeX-Template/blob/master/20240619_SQLParser_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlglot
from datetime import datetime, timedelta
import time
import os

from sqlglot import parse_one, exp, optimizer
from sqlglot.optimizer import optimize
from sqlglot.optimizer.scope import traverse_scope
from sqlglot.errors import ParseError, OptimizeError

<h1> Connection to DB </h1>
<a id="libraries"></a>

<h1> Libraries </h1>
<a id="libraries"></a>

In [ ]:
#cx_Oracle.init_oracle_client(lib_dir=r"C:\oracle\instantclient_21_9")
#cx_Oracle.clientversion()

In [ ]:
#conn = cx_Oracle.connect(user="USER",
#                         password="PSW",
#                         dsn="DSN")

In [ ]:
#statementSQL = """ SELECT a.sql_text, b.session_id, b.session_serial#,c.username, b.sql_id, b.program, b.sample_time
#                   FROM (
#                        SELECT DISTINCT session_id, session_serial#, user_id, sql_id, program, sample_time
#                        FROM DBA_HIST_ACTIVE_SESS_HISTORY WHERE user_id NOT IN (0,9,54,199) AND sql_opcode = 3 --AND sample_time > sysdate-300
#                        ) b
#                   INNER JOIN DBA_HIST_SQLTEXT a ON a.sql_id = b.sql_id
#                   INNER JOIN DBA_USERS c ON b.user_id = c.user_id
#                   ORDER BY b.user_id
#             """

<h1> Import Dataset </h1>
<a id="import"></a>
To run the code without needing to connect to a database, I loaded a dataset with test queries

In [ ]:
# Generate sample data
data = {
    'SQL_TEXT': [
        'SELECT * FROM employees WHERE employee_id = 101',
        'SELECT * FROM employees WHERE employee_id = 102',
        'SELECT * FROM employees WHERE employee_id = 103',
        'SELECT * FROM employees WHERE employee_id = 104',
        'SELECT * FROM departments WHERE department_id = 201',
        'SELECT * FROM departments WHERE department_id = 202',
        'SELECT * FROM departments WHERE department_id = 203',
        'SELECT * FROM departments WHERE department_id = 204',
        'SELECT * FROM projects WHERE project_id = 301',
        'SELECT * FROM projects WHERE project_id = 302'
    ],
    'SESSION_ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'SESSION_SERIAL#': [101, 102, 103, 104, 105, 106, 107, 108, 109, 110],
    'USERNAME': ['user1', 'user2', 'user3', 'user4', 'user5', 'user6', 'user7', 'user8', 'user9', 'user10'],
    'SQL_ID': ['sql1', 'sql2', 'sql3', 'sql4', 'sql5', 'sql6', 'sql7', 'sql8', 'sql9', 'sql10'],
    'PROGRAM': ['program1', 'program2', 'program3', 'program4', 'program5', 'program6', 'program7', 'program8', 'program9', 'program10'],
    'SAMPLE_TIME': [datetime.now() - timedelta(days=i) for i in range(10)]
}

In [ ]:
# Create DataFrame
df_statementSQL = pd.DataFrame(data)
df_statementSQL

,SQL_TEXT,SESSION_ID,SESSION_SERIAL#,USERNAME,SQL_ID,PROGRAM,SAMPLE_TIME
0,SELECT * FROM employees WHERE employee_id = 101,1,101,user1,sql1,program1,2024-06-19 17:56:13.203646
1,SELECT * FROM employees WHERE employee_id = 102,2,102,user2,sql2,program2,2024-06-18 17:56:13.203666
2,SELECT * FROM employees WHERE employee_id = 103,3,103,user3,sql3,program3,2024-06-17 17:56:13.203670
3,SELECT * FROM employees WHERE employee_id = 104,4,104,user4,sql4,program4,2024-06-16 17:56:13.203671
4,SELECT * FROM departments WHERE department_id ...,5,105,user5,sql5,program5,2024-06-15 17:56:13.203673
5,SELECT * FROM departments WHERE department_id ...,6,106,user6,sql6,program6,2024-06-14 17:56:13.203675
6,SELECT * FROM departments WHERE department_id ...,7,107,user7,sql7,program7,2024-06-13 17:56:13.203677
7,SELECT * FROM departments WHERE department_id ...,8,108,user8,sql8,program8,2024-06-12 17:56:13.203678
8,SELECT * FROM projects WHERE project_id = 301,9,109,user9,sql9,program9,2024-06-11 17:56:13.203680
9,SELECT * FROM projects WHERE project_id = 302,10,110,user10,sql10,program10,2024-06-10 17:56:13.203682


In [ ]:
print(df_statementSQL.shape[0]) # Number of rows
print(df_statementSQL.columns.tolist()) # Column names
print(df_statementSQL.dtypes) # Data types

10
['SQL_TEXT', 'SESSION_ID', 'SESSION_SERIAL#', 'USERNAME', 'SQL_ID', 'PROGRAM', 'SAMPLE_TIME']
SQL_TEXT                   object
SESSION_ID                  int64
SESSION_SERIAL#             int64
USERNAME                   object
SQL_ID                     object
PROGRAM                    object
SAMPLE_TIME        datetime64[ns]
dtype: object


<h1> Data Wrangling </h1>
<a id="import"></a>

In [ ]:
# drop rows with null values in column 'SQL_TEXT'
df_statementSQL = df_statementSQL[df_statementSQL['SQL_TEXT'].notna()]

In [ ]:
# Converti da LOB Oracle Object a String
df_statementSQL['SQL_TEXT'] = df_statementSQL['SQL_TEXT'].astype(str) + ' ;'
df_statementSQL['SQL_TEXT'].replace(" SELECT.", "SELECT", inplace=True)
#df_statementSQL['date'] = df_statementSQL['SAMPLE_TIME']
#df_statementSQL['date'] = pd.to_datetime(df_statementSQL['date']).dt.date
df_statementSQL['USERNAME'] = df_statementSQL['USERNAME'].astype(str)

In [ ]:
df_statementSQL.drop(['SESSION_SERIAL#',  'PROGRAM'], axis=1, inplace = True)

<h1> SQL Parser </h1>
<a id="import"></a>

## Functions

In [ ]:
# Get the current time in seconds since the Unix epoch
current_time = time.time()
# Convert the time to a date string in the format YYYY-MM-DD
today_date = time.strftime('%d%m%Y', time.localtime(current_time))

In [ ]:
def remove_substring(string):
    list_to_remove = ['#']
    #list_to_remove = ["@PSDHX1_DBLINK", "@PSPSSX1_DBLINK", "#", "@PSDHX1_REL"]

    string = string.upper()

    for substring in list_to_remove:
        while substring in string:
            string = string.replace(substring, "")

    while "*+" in string: #needed due to incorrect usage of hints
        string = string.replace("*+", "*")

    return string

In [ ]:
def parse_statement(sql_query):

    ast = parse_one(sql_query, read='oracle')
    ast = qualify_columns(ast, schema = None)

    #physical_columns = [(scope.sources.get(c.table).name, c.name)for scope in traverse_scope(ast)for c in scope.columnsif isinstance(scope.sources.get(c.table), exp.Table)]
    physical_columns = []

    for scope in traverse_scope(ast):
        for c in scope.columns:
            if c.table:
                if isinstance(scope.sources.get(c.table), exp.Table):
                    physical_columns.append((scope.sources.get(c.table).name, c.name))

    return physical_columns

## Grouping SQL Parsed

In [ ]:
sql_text_parsed = []

for index, row in df_statementSQL.iterrows():
    query = remove_substring(row['SQL_TEXT'])
    try:
        sql_text_parsed.append(parse_statement(query))
    except ParseError as e:
        sql_text_parsed.append(e)
    continue

df_statementSQL["SQL_TEXT_PARSED"] = sql_text_parsed

In [ ]:
df_statementSQL

,SQL_TEXT,SESSION_ID,USERNAME,SQL_ID,SAMPLE_TIME,SQL_TEXT_PARSED
0,SELECT * FROM employees WHERE employee_id = 101 ;,1,user1,sql1,2024-06-19 17:56:13.203646,"[(EMPLOYEES, EMPLOYEE_ID)]"
1,SELECT * FROM employees WHERE employee_id = 102 ;,2,user2,sql2,2024-06-18 17:56:13.203666,"[(EMPLOYEES, EMPLOYEE_ID)]"
2,SELECT * FROM employees WHERE employee_id = 103 ;,3,user3,sql3,2024-06-17 17:56:13.203670,"[(EMPLOYEES, EMPLOYEE_ID)]"
3,SELECT * FROM employees WHERE employee_id = 104 ;,4,user4,sql4,2024-06-16 17:56:13.203671,"[(EMPLOYEES, EMPLOYEE_ID)]"
4,SELECT * FROM departments WHERE department_id ...,5,user5,sql5,2024-06-15 17:56:13.203673,"[(DEPARTMENTS, DEPARTMENT_ID)]"
5,SELECT * FROM departments WHERE department_id ...,6,user6,sql6,2024-06-14 17:56:13.203675,"[(DEPARTMENTS, DEPARTMENT_ID)]"
6,SELECT * FROM departments WHERE department_id ...,7,user7,sql7,2024-06-13 17:56:13.203677,"[(DEPARTMENTS, DEPARTMENT_ID)]"
7,SELECT * FROM departments WHERE department_id ...,8,user8,sql8,2024-06-12 17:56:13.203678,"[(DEPARTMENTS, DEPARTMENT_ID)]"
8,SELECT * FROM projects WHERE project_id = 301 ;,9,user9,sql9,2024-06-11 17:56:13.203680,"[(PROJECTS, PROJECT_ID)]"
9,SELECT * FROM projects WHERE project_id = 302 ;,10,user10,sql10,2024-06-10 17:56:13.203682,"[(PROJECTS, PROJECT_ID)]"


## Grouping per Tables

In [ ]:
def is_parse_error(obj): return isinstance(obj, ParseError)
filter = df_statementSQL['SQL_TEXT_PARSED'].apply(is_parse_error)

df_ParseError = df_statementSQL[filter]
df_statementSQL = df_statementSQL[~filter]
df_statementSQL['SQL_TEXT_PARSED'] = df_statementSQL['SQL_TEXT_PARSED'].apply(lambda x: list(set(x)))

In [ ]:
results = []

# Ciclo for per scorrere ogni riga del dataframe df
for i, row in df_statementSQL.iterrows():
    # Ciclo for per scorrere ogni elemento nella colonna 'sql_text_parsed' della riga corrente
    for j in row['SQL_TEXT_PARSED']:
        # Aggiunta di una tupla a `results` che contiene i valori della colonna 'user' e 'date'
        # e i valori j[0] e j[1] delle tuple presenti in 'SQL_TEXT_PARSED'
        results.append((row['USERNAME'],row['SQL_TEXT'], j[0], j[1], row['SAMPLE_TIME']))


In [ ]:
# Creazione del df che contiene un record per ogni occorrenza di tabella e colonna presente x user,date e SQL statement
result_df_withSQL = pd.DataFrame(results, columns=['user', 'sql_text', 'table', 'column', 'date' ])

In [ ]:
result_df_withSQL

,user,sql_text,table,column,date
0,user1,SELECT * FROM employees WHERE employee_id = 101 ;,EMPLOYEES,EMPLOYEE_ID,2024-06-19 17:56:13.203646
1,user2,SELECT * FROM employees WHERE employee_id = 102 ;,EMPLOYEES,EMPLOYEE_ID,2024-06-18 17:56:13.203666
2,user3,SELECT * FROM employees WHERE employee_id = 103 ;,EMPLOYEES,EMPLOYEE_ID,2024-06-17 17:56:13.203670
3,user4,SELECT * FROM employees WHERE employee_id = 104 ;,EMPLOYEES,EMPLOYEE_ID,2024-06-16 17:56:13.203671
4,user5,SELECT * FROM departments WHERE department_id ...,DEPARTMENTS,DEPARTMENT_ID,2024-06-15 17:56:13.203673
5,user6,SELECT * FROM departments WHERE department_id ...,DEPARTMENTS,DEPARTMENT_ID,2024-06-14 17:56:13.203675
6,user7,SELECT * FROM departments WHERE department_id ...,DEPARTMENTS,DEPARTMENT_ID,2024-06-13 17:56:13.203677
7,user8,SELECT * FROM departments WHERE department_id ...,DEPARTMENTS,DEPARTMENT_ID,2024-06-12 17:56:13.203678
8,user9,SELECT * FROM projects WHERE project_id = 301 ;,PROJECTS,PROJECT_ID,2024-06-11 17:56:13.203680
9,user10,SELECT * FROM projects WHERE project_id = 302 ;,PROJECTS,PROJECT_ID,2024-06-10 17:56:13.203682


In [ ]:
# Group the data by user and table, count the unique values in the sql_text column
result_df_withSQL_grouped = result_df_withSQL.groupby(['user','table'])['sql_text'].nunique()
# Create a new DataFrame from the grouped result
result_df_withSQL_grouped = result_df_withSQL_grouped.reset_index(name='counts')
# df con raggruppamento in ordine descrescente x utente ed occorrenze
result_df_withSQL_grouped.sort_values(by=['counts','user'], ascending=False, inplace=True)

result_df_withSQL_grouped.head()

,user,table,counts
9,user9,PROJECTS,1
8,user8,DEPARTMENTS,1
7,user7,DEPARTMENTS,1
6,user6,DEPARTMENTS,1
5,user5,DEPARTMENTS,1


## Grouping per Columns

In [ ]:
# Eliminazione della colonna SQL Statement
result_df = result_df_withSQL.drop(['sql_text'], axis=1)

# Raggruppa i dati in base a "user", "date" e "column"
result_df_grouped = result_df_withSQL.groupby(['user','column', 'table'])['sql_text'].nunique()

# Conta le occorrenze di ogni gruppo
result_df_grouped = result_df_grouped.reset_index(name='counts')

# df con raggruppamento in ordine descrescente x utente ed occorrenze
result_df_grouped.sort_values(by=['counts', 'user'], ascending=False, inplace=True)

In [ ]:
result_df_grouped.head()

,user,column,table,counts
9,user9,PROJECT_ID,PROJECTS,1
8,user8,DEPARTMENT_ID,DEPARTMENTS,1
7,user7,DEPARTMENT_ID,DEPARTMENTS,1
6,user6,DEPARTMENT_ID,DEPARTMENTS,1
5,user5,DEPARTMENT_ID,DEPARTMENTS,1


<h1> Exporting </h1>
<a id="export"></a>

In [ ]:
# Define the file paths and names
file_path = 'outputs/'
tables_file_name = f'{file_path}{today_date}_tablesOccurences.csv'
columns_file_name = f'{file_path}{today_date}_columnsOccurences.csv'
columnsTables_list_name = f'{file_path}{today_date}_columnsTables.csv'


try:
    # Check if the files already exist
    tables_file_exists = os.path.isfile(tables_file_name)
    columns_file_exists = os.path.isfile(columns_file_name)
    columnsTables_list_exists = os.path.isfile(columnsTables_list_name)

    # Notify the user if the files will be overwritten
    if tables_file_exists or columns_file_exists or columnsTables_list_exists:
        print('Warning: One or more output files already exist and has been overwritten!')

    # Save the result dataframes to CSV files
    result_df_withSQL_grouped.to_csv(tables_file_name, index=False)
    result_df_grouped.to_csv(columns_file_name, index=False)
    # df_colonneTabelle.to_csv(columnsTables_list_name, index=False)
    print('Data saved successfully!')
except Exception as e:
    # Handle any exceptions that occurred during saving
    error_message = f'Error occurred while saving data to CSV files: {str(e)}'
    print(error_message)
    # Remove any partially saved files
    if os.path.isfile(tables_file_name):
        os.remove(tables_file_name)
    if os.path.isfile(columns_file_name):
        os.remove(columns_file_name)
    if os.path.isfile(columnsTables_list_name):
        os.remove(columnsTables_list_name)

Error occurred while saving data to CSV files: Cannot save file into a non-existent directory: 'outputs'
